In [1]:
import pandas as pd
import xml.etree.ElementTree as et
import xml.dom.minidom as md

In [2]:
# Récuperation du fichier dans Google Colab
filename = './drive/MyDrive/base_ratp.csv'

# Lecture du fichier CSV
df = pd.read_csv(filename, delimiter=';') 

In [3]:
# Tri du dataset
df = df.sort_values(by=['service_id', 'direction_id', 'route_id', 'stop_sequence'])

In [4]:
# Construction de la liste des lignes de métro
line_list = list(df['service_short_name'].unique())
print(line_list)

['2', '1', '3', '3B', '4', '5', '6', '7', '7B', '8', '9', '10', '11', '12', '13', '14']


In [5]:
# Construction d'un dictionnaire où les clés sont les noms des lignes et les valeurs les dataframes corespondants
df_by_line = {}

for line in line_list:
    df_by_line[line] = df[df.service_short_name == line]

In [6]:
# Retourne le nombre de station sur la ligne
def nb_stations(dataframe):
    return len(dataframe['station_id'].unique())

In [7]:
# Retourne vrai si la ligne n'a pas de bifurcation
def is_station_straight(dataframe):
    return len(dataframe) == len(dataframe['route_id'].unique()) * len(dataframe['station_id'].unique())

In [8]:
# Construit le XML correspondant à une simple liste de station
def list_to_XML(stations, element):
    for station in stations:
            info, changes = station[0], station[1]
            s = et.SubElement(element, "station", id=str(info['station_id']), lat=str(info['station_lat']), lon=str(info['station_lon']))
            n = et.SubElement(s, 'name')
            n.text = info['station_name']
            d = et.SubElement(s, 'desc')
            d.text = info['station_desc']

            if changes:
                c = et.SubElement(s, 'changes')
                for change in changes:
                    cl = et.SubElement(c, 'changeline')
                    cl.text = change


In [9]:
# Retourne le XML correspondant à une ligne à partir du dataframe de la ligne
def get_stations(dataframe):
    line_name = dataframe.iloc[0].service_short_name
    df_line = dataframe.drop_duplicates(subset='station_id')

    l = et.SubElement(root, "line", name=line_name)

    # Cas où la ligne n'a pas de bifurcation
    if is_station_straight(dataframe):
        
        # Construction de la liste des stations
        line = []
        for station in df_line.iterrows():
            change = list(set(df[df.station_id == station[1]['station_id']].service_short_name))
            change.remove(line_name)
            line.append((station[1], change))
        
        # Contruction du XML de la ligne
        path = et.SubElement(l, "path")
        list_to_XML(line, path)
        return          
    
    # Cas où la ligne a une bifurcation
    else:
        nb_routes = len(dataframe.drop_duplicates(subset='route_id'))
        
        # Cas où la ligne a une bifurcation interne (2 terminus)
        if nb_routes == 2:
            
            # Construction des listes des stations
            first_list = []
            bottom_bifur = []
            top_bifur = []
            last_list = []
            
            first_done = False
            bottom_done = False
            
            for station in df_line.iterrows():
                change = list(set(df[df.station_id == station[1]['station_id']].service_short_name))
                change.remove(line_name)

                if len(dataframe[dataframe.station_id == station[1]['station_id']]) == 2:
                    if first_done:
                        bottom_done = True
                        last_list.append((station[1], change))
                    else :
                        first_list.append((station[1], change))
                
                else:
                    first_done = True
                    if bottom_done:
                        top_bifur.append((station[1], change))
                    else:
                        bottom_bifur.append((station[1], change))
            
            # Contruction du XML de la ligne
            path = et.SubElement(l, "path")
            list_to_XML(first_list, path)
            bifur = et.SubElement(l, "bifur")
            path = et.SubElement(bifur, "subpath")
            list_to_XML(bottom_bifur, path)
            path = et.SubElement(bifur, "subpath")
            list_to_XML(list(reversed(top_bifur)), path)
            path = et.SubElement(l, "path")
            list_to_XML(last_list, path)
            return
           
    
        # Cas où la ligne a une bifurcation qui ne se rejoint pas (3 terminus)
        if (nb_routes == 4):
            
            # Construction des listes des stations
            main_route = dataframe.iloc[0]['route_id']

            main_list = []
            bottom_bifur = []
            top_bifur = []

            for station in df_line.iterrows():
                change = list(set(df[df.station_id == station[1]['station_id']].service_short_name))
                change.remove(line_name)

                if len(dataframe[dataframe.station_id == station[1]['station_id']]) == 4:
                    main_list.append((station[1], change))
                else:
                    if station[1]['route_id'] == main_route:
                        bottom_bifur.append((station[1], change))
                    else:
                        top_bifur.append((station[1], change))

            # Contruction du XML de la ligne
            path = et.SubElement(l, "path")
            list_to_XML(main_list, path)
            bifur = et.SubElement(l, "bifur")
            path = et.SubElement(bifur, "subpath")
            list_to_XML(bottom_bifur, path)
            path = et.SubElement(bifur, "subpath")
            list_to_XML(top_bifur, path)
            return


In [10]:
# Construction du XML et écriture du fichier
root = et.Element("root")
for line in line_list:
    get_stations(df_by_line[line])

with open("base_ratp.xml", "w") as f:
    f.write(str(md.parseString(et.tostring(root)).toprettyxml(indent="   ")))